# RBP Maps 0.2 can be used as a reference for SE events
- This notebook verifies the changes made do not affect the last time dataframes were checked independently. 

In [1]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import os
import sys
import math

sys.path.insert(0, '/home/bay001/projects/codebase/rbp-maps/maps/')
from pandas.util.testing import assert_frame_equal
from density import ReadDensity
from density import normalization_functions as norm
from density import intervals
from density import Feature
pd.set_option('max_colwidth', 1000)
pd.set_option('display.max_rows', 1500)

In [2]:
wd = '/home/bay001/projects/codebase/temp/'
outfile = '/home/bay001/projects/codebase/temp/out/rbfox2.svg'
ref_wd = '/home/bay001/projects/maps_20160420/analysis/tests/for_eric/'
read_nums = '/home/bay001/projects/codebase/data/ALLDATASETS_submittedonly.txt.mapped_read_num'
ipbam = '/home/bay001/projects/codebase/temp/204_01_RBFOX2.merged.r2.bam'
inpbam = '/home/bay001/projects/codebase/temp/RBFOX2-204-INPUT_S2_R1.unassigned.adapterTrim.round2.rmRep.rmDup.sorted.r2.bam'
ref_positive = '/home/bay001/projects/codebase/temp/RBFOX2-HepG2-SE-positive.txt'
ref_negative = '/home/bay001/projects/codebase/temp/RBFOX2-HepG2-SE-negative.txt'
ref_bg = '/home/bay001/projects/codebase/temp/RBFOX2-HepG2-SE.txt'
program = '/home/bay001/projects/codebase/rbp-maps/maps/plot_density.py'
readnums = pd.read_table(read_nums,names=['rbp','reads'])


In [3]:
def make():
    ref1 = pd.read_table('/projects/ps-yeolab3/bay001/maps/annotations/RBFOX2-HepG2-SE.txt')
    ref1 = ref1[ref1['IncLevelDifference']>0.1]
    ref1.to_csv('/home/bay001/projects/codebase/temp/RBFOX2-HepG2-SE-positive.txt',sep='\t',index=None) 

def diff(df1, df2):
    try:
        assert_frame_equal(df1, df2)
        return('same')
    except Exception as e:  # appeantly AssertionError doesn't catch all
        return('{} different'.format(e))
def round_nearest(x, a):
    return round(round(x / a) * a, -int(math.floor(math.log10(a))))


In [9]:
! python $program -ip $ipbam -input $inpbam -a $ref_positive $ref_negative $ref_bg --annotation_type rmats rmats rmats -o $outfile -u -e SE

('topdir', '/home/bay001/projects/codebase/rbp-maps/maps/plot_density.py')
No handlers could be found for logger "plot_features"


In [ ]:
cmd = 'python {} -ip {} -input {} -c {} {} {} -at {} {} {} -o {} -e {}'.format(
    program, ipbam, inpbam, ref_positive, ref_negative, ref_bg, 'rmats', 'rmats', 'rmats', outfile, 'se'
)
cmd

### Test input raw matrix

In [ ]:
ref_input_raw_matrix = 'included.input.204_01_RBFOX2.included.feature.se.raw_density_matrix.csv'
ref_input = pd.read_table(
    os.path.join(ref_wd,ref_input_raw_matrix),
    sep=',',
    index_col=0
)
test_input_raw_matrix = '/home/bay001/projects/codebase/temp/out/rbfox2.RBFOX2-HepG2-SE-positive.input.raw_density.csv'
test_input = pd.read_table(test_input_raw_matrix,sep=',',
    index_col=0)

diff(test_input, ref_input)

### Test ip raw matrix

In [ ]:
# test ip
ref_ip_raw_matrix = 'included.ip.204_01_RBFOX2.included.feature.se.raw_density_matrix.csv'
ref_ip = pd.read_table(
    os.path.join(ref_wd,ref_ip_raw_matrix),
    sep=',',
    index_col=0
)
test_ip_raw_matrix = '/home/bay001/projects/codebase/temp/out/rbfox2.RBFOX2-HepG2-SE-positive.ip.raw_density.csv'
test_ip = pd.read_table(test_ip_raw_matrix,sep=',',
    index_col=0)

diff(test_ip, ref_ip)

# test ip pdf function

In [ ]:
test_ip_pdf = norm.calculate_pdf(test_ip)
ref_ip_pdf = pd.read_table('/home/bay001/projects/maps_20160420/analysis/tests/for_eric/ip_pdf.csv',sep=',',index_col=0)
diff(test_ip_pdf,ref_ip_pdf)

# test input pdf function

In [ ]:
test_input_pdf = norm.calculate_pdf(test_input)
ref_input_pdf = pd.read_table('/home/bay001/projects/maps_20160420/analysis/tests/for_eric/input_pdf.csv',sep=',',index_col=0)
diff(test_input_pdf,ref_input_pdf)

# Test normalization (subtract by region) function
- caveats: see below

In [ ]:

# ip reads: 4275451
# input reads: 3765762


test_norm = norm.normalize_and_per_region_subtract(test_ip, test_input,1000000.0/4275451,1000000.0/3765762,0)
ref_norm = pd.read_table('/home/bay001/projects/maps_20160420/analysis/tests/for_eric/included.204_01_RBFOX2.subtract_by_region.feature.normed_matrix.csv',
                        sep=',',index_col=0)

merged = pd.merge(ref_norm, test_norm, how='left', left_index=True, right_index=True)

test_norm = test_norm.ix[merged.index]
test_norm.columns = test_norm.columns + '_TEST'

ref_norm = ref_norm.ix[merged.index]
ref_norm.columns = ref_norm.columns + '_REF'

print(test_norm.shape)
print(ref_norm.shape)

In [ ]:
merged = pd.merge(ref_norm, test_norm, how='left', left_index=True, right_index=True)
merged.shape

##### diff(test_norm, ref_norm)
- This will likely return false due to the rounding issues. When the ref was written to file, it rounded off to the nearest 14th digit. However the normalization function will not round at all, so we need to manually do this to check for equality. 
- Somehow, the reference was truncated. ARGH!
- but the good news is that it is all matching up to the truncation. 

In [ ]:
merged.head()

In [ ]:

for c in range(0,1400):
    for i in merged.index:
        if i != ix_in_question: # we don't need to check the index that was truncated. 
            t = round_nearest(merged.ix[i]['{}_TEST'.format(c)],13)
            r = round_nearest(merged.ix[i]['{}_REF'.format(c)],13)
            if(r != t):
                if np.isnan(r) and np.isnan(r):
                    pass
                else:
                    print(i, round_nearest(
                            merged.ix[i]['{}_TEST'.format(col)],13), 
                          round_nearest(
                            merged.ix[i]['{}_REF'.format(col)],13))


# remake test files...
- because sometimes you're really dumb and delete them.

In [ ]:
alldata = pd.read_table('/home/bay001/projects/codebase/temp/RBFOX2-HepG2-SE.txt',index_col=0)

In [ ]:
alldata

In [ ]:
original == better

In [ ]:
original